In [2]:
# Declaring path variables to make Basemap work in jupyter hub. Make sure that the conda environment path matches yours
import os
os.environ['PATH'] = '/home/jovyan/my-conda-envs/qiime2-2019.1/bin:' + os.environ['PATH']
os.environ['PROJ_LIB'] = '/home/jovyan/my-conda-envs/qiime2-2019.1/share/proj'

In [3]:
# General Tool Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
import collections
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
import math

from biota.database import (get_samples,
                            get_sample_data,
                            get_pg_connection,
                            get_data)
from biota.util import (rarefy,
                        make_tree,
                        get_dms,
                        filter_dm_and_map,
                        get_tax_table)
from IPython.core.display import display, HTML
from pickle import dump, load

import lasio

# SQL Query imports
from sqlalchemy import create_engine, engine, inspect

from functools import partial

from ipyparallel import Client
from sourcetracker import gibbs

# Clustering imports
from scipy.spatial.distance import pdist
from scipy.cluster import hierarchy
from skbio.stats.ordination import pcoa

from biota.util import assign_formations


%matplotlib inline

In [4]:
waterpd = pd.read_excel('Foraker Water Rates Update 20191011 Local.xlsx')

In [5]:
AllData = pd.read_excel('Foraker Production 10-14 Check v3.xlsx')

In [71]:
#waterpd.head(10)

In [32]:
waterpd = waterpd.set_index('Date Time').copy()

In [72]:
#waterpd.head()

In [16]:
colnames = waterpd.columns

In [49]:
#waterpd['FORAKER 1207 9LMH-11'].head(20)

In [171]:
welldfs = {}
namdict = []
for i,well in enumerate(colnames):
    match = {i:well}
    namdict.append(match)
    welldfs[i] = waterpd[[well]].copy()
    welldfs[i] = welldfs[i].reset_index()
    welldfs[i]['Well'] = well
    wellname = well
    welldfs[i].columns = {'Date','GROSS WTR','X'}

In [177]:
welldfs[5].head()

,Date,X,GROSS WTR
0,2019-03-23,NaN,FORAKER 1207 5WH-11
1,2019-03-24,NaN,FORAKER 1207 5WH-11
2,2019-03-25,NaN,FORAKER 1207 5WH-11
3,2019-03-26,NaN,FORAKER 1207 5WH-11
4,2019-03-27,NaN,FORAKER 1207 5WH-11


In [178]:
dflist = []
for i,well in enumerate(welldfs):
    item = welldfs[well]
    dflist.append(item)

In [142]:
soso = welldfs[10].copy()
soso2 = welldfs[9].copy()

In [164]:
#dflist = [soso,soso2]

In [179]:
cpd = pd.concat(dflist)

In [186]:
cpd.to_csv('../WaterRates.csv')

In [157]:
#NewWaters = pd.concat(welldfs,ignore_index=True, sort=False)

In [158]:
#NewWaters.head()

In [82]:
welldfs[3].columns = {'A','B'}

In [93]:
welldfs[0].head()

,GROSS WTR,FORAKER 1207 1SMH-11
Date Time,,
2019-03-23,NaN,FORAKER 1207 1SMH-11
2019-03-24,NaN,FORAKER 1207 1SMH-11
2019-03-25,NaN,FORAKER 1207 1SMH-11
2019-03-26,1166.60,FORAKER 1207 1SMH-11
2019-03-27,2200.91,FORAKER 1207 1SMH-11


In [187]:
Julypd = pd.read_excel('ForakerProductionDataJulyV2.xlsx')

In [ ]:
#Read updated water rates
Julypd = pd.read_excel('ForakerProductionDataJulyV2.xlsx')

In [7]:
#Group original data by well
CDpd = AllData.groupby('Well')
CDnames = AllData.Well.unique()


In [188]:
waterpd = pd.read_csv('WaterRatesCompiled.csv')

In [191]:
prod = Julypd.copy()

In [201]:
groupedwells = prod.groupby('Well')

In [283]:
fifi11 = groupedwells.get_group('FORAKER 1207 1SMH-11').copy()

In [199]:
prod['Recon WTR'] = 0

In [204]:
watergroups = waterpd.groupby('Well')


In [279]:
soso = watergroups.get_group('FORAKER 1207 1SMH-11').copy()

In [276]:
wellnames = prod.Well.unique()
waternames = waterpd.Well.unique()

In [324]:
newdfs = {}
#import datetime
for i,well in enumerate(groupedwells):
    wellname = wellnames[i]
    waterwell = watergroups.get_group(wellname).copy()
    wellpd = groupedwells.get_group(wellname).copy()
    waterwell['PERIOD'] = pd.to_datetime(waterwell['PERIOD'])
    waterdic = waterwell.set_index('PERIOD')['GROSS WTR'].to_dict()
    wellpd['RECON WTR'] = wellpd['PERIOD'].map(waterdic)
    wellpd['RECON WCT'] = 100*wellpd['RECON WTR'] / (wellpd['RECON WTR']+wellpd['GROSS OIL'])
    wellpd ['WCT'] = 100*wellpd['GROSS WTR'] / (wellpd['GROSS WTR']+wellpd['GROSS OIL'])
    wellpd ['Liq Rate'] = wellpd['GROSS WTR']+wellpd['GROSS OIL']
    wellpd ['RECON Liq Rate'] = wellpd['RECON WTR']+wellpd['GROSS OIL']
    newdfs[i]=wellpd

In [325]:
newdfs[10].head()

,Well,PERIOD,TP,CP,BH,GROSS OIL,GROSS WTR,GROSS GAS,GROSS BOE,Downtime,...,Days Online,Formation,Pad,Comp Order,Recon WTR,RECON WTR,RECON WCT,WCT,Liq Rate,RECON Liq Rate
NaN,FORAKER 1207 11SMH-11,2019-04-05,0.0,1268.0,NaN,366.92,664.0,172.0,395.5867,0.0,...,1,SM,Pad1,6Z,0,664.34,64.420224,64.408490,1030.92,1031.26
NaN,FORAKER 1207 11SMH-11,2019-04-06,0.0,1328.0,NaN,718.20,2176.0,1108.0,902.8667,0.0,...,2,SM,Pad1,6Z,0,2196.67,75.360822,75.184852,2894.20,2914.87
NaN,FORAKER 1207 11SMH-11,2019-04-07,0.0,1247.0,NaN,412.89,2525.0,1678.0,692.5567,0.0,...,3,SM,Pad1,6Z,0,2525.09,85.946467,85.946036,2937.89,2937.98
NaN,FORAKER 1207 11SMH-11,2019-04-08,0.0,1247.0,NaN,631.48,3212.0,1576.0,894.1467,0.0,...,4,SM,Pad1,6Z,0,2018.82,76.173263,83.570098,3843.48,2650.30
NaN,FORAKER 1207 11SMH-11,2019-04-09,0.0,1219.0,NaN,755.55,2116.0,1890.0,1070.5500,0.0,...,5,SM,Pad1,6Z,0,2018.82,72.766790,73.688426,2871.55,2774.37


In [326]:
Alldfs = pd.concat(newdfs)

In [327]:
Alldfs.to_csv('../Practice/UpdatedForakerWTR V3.csv')

In [310]:
#fifi = prod.merge(waterpd[['Well']])